In [ ]:
import socket
socket.gethostname()

In [ ]:
import tensorflow as tf
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, Adam

tf.test.is_built_with_cuda(), tf.test.is_gpu_available(),  tf.__version__

In [ ]:
class MnistData():
    
    def __init__(self):
        (train_images, self._train_labels), (test_images, self._test_labels) = tf.keras.datasets.mnist.load_data()
        
        self.shape = train_images[0].shape
        self.num_classes = 10

        self.train_images = train_images.astype('float32') / 255.0
        self.test_images = test_images.astype('float32') / 255.0
        
        self.train_labels = keras.utils.to_categorical(self._train_labels, self.num_classes)
        self.test_labels = keras.utils.to_categorical(self._test_labels, self.num_classes)
        
        if tf.test.is_built_with_cuda(): 
            self.data_format = 'channels_first'
        else:
            self.data_format = 'channels_last'

    def target_shape(self):
        shape = self.shape
        if tf.test.is_built_with_cuda():
            return (1, ) + shape
        else:
            return shape + (1, )
        
    def get_label(self, index, test=True):
        if test:
            return self._test_labels[index]
        else:
            return self._train_labels[index]
    
    def show_image(self, index, test=True):
        if test:
            image = self.test_images[index]
        else:
            image = self.train_images[index]
            
        p = figure(x_range=(0, self.img_rows), y_range=(0, self.img_cols), width=280, height=280)
        p.image([np.flipud(image)], x=0, y=0, dw=self.img_rows, dh=self.img_cols)
        show(p)     

In [ ]:
from databrickslabs_jupyterlab.acc_loss_plot import VisualModel

class MnistModel(VisualModel):
    
    def __init__(self, data, batch_size, epochs):
        self.data = data
        self.batch_size = batch_size
        self.epochs = epochs
        self.t_shape = self.data.target_shape()
        self.i_shape = self.data.shape
        self.format = self.data.data_format
        self.classes = self.data.num_classes
        
    def create_model(self, lr, beta_1, beta_2, epsilon):
        self.model = Sequential((            
            Reshape(target_shape=self.t_shape, input_shape=self.i_shape),
            Conv2D(filters=24, kernel_size=6, activation="relu", data_format=self.format),
            Dropout(0.5),
            Conv2D(filters=48, kernel_size=5, activation="relu", data_format=self.format),
            Dropout(0.5),
            Conv2D(filters=64, kernel_size=4, activation="relu", data_format=self.format),
            Dropout(0.5),
            Flatten(),
            Dense(200, activation='relu'),
            Dropout(0.5),
            Dense(self.classes, activation='softmax'),
        ))

        self.model.compile(loss=categorical_crossentropy, 
                           optimizer=Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon), 
                           metrics=['accuracy'])
        return self

    def fit(self, callbacks=None, verbose=0):
        try:
            self.model.fit(
                self.data.train_images, 
                self.data.train_labels,
                batch_size=self.batch_size,
                epochs=self.epochs,
                verbose=verbose,
                validation_data=(self.data.test_images, self.data.test_labels), 
                callbacks=(callbacks or []))
        except KeyboardInterrupt:
            print("\n*** INTERRUPTED ***\n")    

In [ ]:
mnistData = MnistData()

In [ ]:
mnist = MnistModel(mnistData, batch_size=100, epochs=20)
mnist\
    .create_model(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)\
    .fit(callbacks=[mnist.callback(min_acc=0.94, max_loss=0.5, table=True, visual=True)])